In [1]:
# Core libraries and model APIs
from pathlib import Path
import warnings

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score, f1_score, classification_report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier



In [2]:
df = pd.read_excel('ctg.xlsx', sheet_name = 'Data', header = 1)
df

,b,e,AC,FM,UC,DL,DS,DP,DR,Unnamed: 9,...,E,AD,DE,LD,FS,SUSP,Unnamed: 42,CLASS,Unnamed: 44,NSP
0,240.0,357.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,-1.0,-1.0,-1.0,-1.0,1.0,-1.0,NaN,9.0,NaN,2.0
1,5.0,632.0,4.0,0.0,4.0,2.0,0.0,0.0,0.0,NaN,...,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,NaN,6.0,NaN,1.0
2,177.0,779.0,2.0,0.0,5.0,2.0,0.0,0.0,0.0,NaN,...,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,NaN,6.0,NaN,1.0
3,411.0,1192.0,2.0,0.0,6.0,2.0,0.0,0.0,0.0,NaN,...,-1.0,1.0,-1.0,-1.0,-1.0,-1.0,NaN,6.0,NaN,1.0
4,533.0,1147.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,2.0,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2124,1576.0,3049.0,1.0,0.0,9.0,0.0,0.0,0.0,0.0,NaN,...,1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,5.0,NaN,2.0
2125,2796.0,3415.0,1.0,1.0,5.0,0.0,0.0,0.0,0.0,NaN,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,1.0,NaN,1.0
2126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2127,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,...,72.0,332.0,252.0,107.0,69.0,197.0,NaN,NaN,NaN,NaN


In [3]:
def clean_data(df: pd.DataFrame):
    """
    Cleans the input dataframe by:
    1. Removing empty rows and columns.
    2. Keeping only the specified features and the target.
    3. Removing duplicate rows.
    4. Dropping rows with missing target values.
    5. Splitting into features (X) and target (y).

    Args:
        df (pd.DataFrame): The raw dataframe.

    Returns:
        X (pd.DataFrame): Feature dataframe (restricted to chosen features).
        y (pd.Series): Target column ('NSP').
    """

    # Step 1: Make a working copy of the dataframe
    cleaned = df.copy()

    # Step 2: Drop any rows or columns that are completely empty
    cleaned = cleaned.dropna(axis=0, how='all').dropna(axis=1, how='all')

    # Step 3: Define target and allowed features
    target_col = 'NSP'
    allowed_features = {
        'b', 'e', 'AC', 'FM', 'UC', 'DL', 'DS', 'DP', 'DR', 'LB',
        'ASTV', 'MSTV', 'ALTV', 'MLTV',
        'Width', 'Min', 'Max', 'Nmax', 'Nzeros', 'Mode',
        'Median', 'Variance', 'Tendency'
    }

    # Step 4: Keep only allowed features + target
    keep_cols = list(allowed_features) + [target_col]
    cleaned = cleaned.loc[:, [c for c in cleaned.columns if c in keep_cols]]

    # Step 5: Remove duplicate rows
    cleaned = cleaned.drop_duplicates()

    # Step 6: Drop rows where target is missing
    cleaned = cleaned.dropna(subset=[target_col])

    # Step 7: Separate features and target
    X = cleaned.drop(columns=[target_col])
    y = cleaned[target_col]

    return X, y


In [4]:
X, y = clean_data(df)
X

,b,e,AC,FM,UC,DL,DS,DP,DR,LB,...,MLTV,Width,Min,Max,Nmax,Nzeros,Mode,Median,Variance,Tendency
0,240.0,357.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,120.0,...,2.4,64.0,62.0,126.0,2.0,0.0,120.0,121.0,73.0,1.0
1,5.0,632.0,4.0,0.0,4.0,2.0,0.0,0.0,0.0,132.0,...,10.4,130.0,68.0,198.0,6.0,1.0,141.0,140.0,12.0,0.0
2,177.0,779.0,2.0,0.0,5.0,2.0,0.0,0.0,0.0,133.0,...,13.4,130.0,68.0,198.0,5.0,1.0,141.0,138.0,13.0,0.0
3,411.0,1192.0,2.0,0.0,6.0,2.0,0.0,0.0,0.0,134.0,...,23.0,117.0,53.0,170.0,11.0,0.0,137.0,137.0,13.0,1.0
4,533.0,1147.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,132.0,...,19.9,117.0,53.0,170.0,9.0,0.0,137.0,138.0,11.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2121,2059.0,2867.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,140.0,...,7.2,40.0,137.0,177.0,4.0,0.0,153.0,152.0,2.0,0.0
2122,1576.0,2867.0,1.0,0.0,9.0,0.0,0.0,0.0,0.0,140.0,...,7.1,66.0,103.0,169.0,6.0,0.0,152.0,151.0,3.0,1.0
2123,1576.0,2596.0,1.0,0.0,7.0,0.0,0.0,0.0,0.0,140.0,...,6.1,67.0,103.0,170.0,5.0,0.0,153.0,152.0,4.0,1.0
2124,1576.0,3049.0,1.0,0.0,9.0,0.0,0.0,0.0,0.0,140.0,...,7.0,66.0,103.0,169.0,6.0,0.0,152.0,151.0,4.0,1.0


In [5]:
X.columns

Index(['b', 'e', 'AC', 'FM', 'UC', 'DL', 'DS', 'DP', 'DR', 'LB', 'ASTV',
       'MSTV', 'ALTV', 'MLTV', 'Width', 'Min', 'Max', 'Nmax', 'Nzeros', 'Mode',
       'Median', 'Variance', 'Tendency'],
      dtype='object')

In [6]:
y

0       2.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2121    2.0
2122    2.0
2123    2.0
2124    2.0
2125    1.0
Name: NSP, Length: 2115, dtype: float64

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# --- Example function to run SMOTE + CV ---
def evaluate_with_smote_cv(X, y, n_splits=5, random_state=42):
    """
    Evaluates a RandomForest model with SMOTE oversampling applied inside CV folds.
    Returns balanced accuracy scores for each fold.
    """

    # Define pipeline: SMOTE applied only to training folds
    pipeline = Pipeline(steps=[
        ('smote', SMOTE(random_state=random_state)),
        ('rf', RandomForestClassifier(
            n_estimators=300,
            class_weight='balanced',
            random_state=random_state,
            n_jobs=-1
        ))
    ])

    # Stratified 5-fold split
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # Use balanced accuracy (good for imbalanced data)
    scorer = make_scorer(balanced_accuracy_score)

    # Cross-validation with SMOTE inside each training fold
    scores = cross_val_score(pipeline, X, y, cv=skf, scoring=scorer, n_jobs=-1)

    print("Balanced Accuracy per fold:", np.round(scores, 3))
    print("Mean Balanced Accuracy:", np.round(np.mean(scores), 3))

    return scores


In [8]:
evaluate_with_smote_cv(X,y)

Balanced Accuracy per fold: [0.895 0.926 0.934 0.867 0.871]
Mean Balanced Accuracy: 0.899


array([0.89527587, 0.92638538, 0.93438428, 0.86730358, 0.87113002])

In [9]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import StratifiedKFold, cross_val_score
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import make_scorer, balanced_accuracy_score
# from imblearn.over_sampling import SMOTE
# from imblearn.pipeline import Pipeline
# import optuna

# # --- Objective function for Bayesian optimization ---
# def objective(trial, X, y, n_splits=5, random_state=42):
#     # Suggest hyperparameters for Random Forest
#     n_estimators = trial.suggest_int('n_estimators', 100, 1000)
#     max_depth = trial.suggest_int('max_depth', 3, 30)
#     min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
#     min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
#     max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2', None])
#     bootstrap = trial.suggest_categorical('bootstrap', [True, False])

#     # Pipeline with SMOTE + RF
#     pipeline = Pipeline(steps=[
#         ('smote', SMOTE(random_state=random_state)),
#         ('rf', RandomForestClassifier(
#             n_estimators=n_estimators,
#             max_depth=max_depth,
#             min_samples_split=min_samples_split,
#             min_samples_leaf=min_samples_leaf,
#             max_features=max_features,
#             bootstrap=bootstrap,
#             class_weight='balanced',
#             random_state=random_state,
#             n_jobs=-1
#         ))
#     ])

#     # Cross-validation
#     skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
#     scorer = make_scorer(balanced_accuracy_score)
#     scores = cross_val_score(pipeline, X, y, cv=skf, scoring=scorer, n_jobs=-1)

#     return np.mean(scores)

# # --- Run Bayesian optimization ---
# def optimize_rf(X, y, n_trials=50, random_state=42):
#     study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=random_state))
#     study.optimize(lambda trial: objective(trial, X, y, random_state=random_state), n_trials=n_trials)

#     print("Best Trial:")
#     print("  Balanced Accuracy:", study.best_value)
#     print("  Params:", study.best_params)

#     return study


In [10]:
# study = optimize_rf(X, y, n_trials=50)  # try 50 trials first
# best_params = study.best_params


In [11]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# --- Updated function with best RF parameters ---
def evaluate_with_smote_cv_rfoptimal(X, y, n_splits=5, random_state=42):
    """
    Evaluates a tuned RandomForest model with SMOTE oversampling applied inside CV folds.
    Returns balanced accuracy scores for each fold.
    """

    # Define pipeline: SMOTE applied only to training folds
    pipeline = Pipeline(steps=[
        ('smote', SMOTE(random_state=random_state)),
        ('rf', RandomForestClassifier(
            n_estimators=437,
            max_depth=29,
            min_samples_split=15,
            min_samples_leaf=6,
            max_features='sqrt',
            bootstrap=True,
            class_weight='balanced',
            random_state=random_state,
            n_jobs=-1
        ))
    ])

    # Stratified k-fold split
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    # Balanced accuracy scorer
    scorer = make_scorer(balanced_accuracy_score)

    # Cross-validation with SMOTE inside each training fold
    scores = cross_val_score(pipeline, X, y, cv=skf, scoring=scorer, n_jobs=-1)

    print("Balanced Accuracy per fold:", np.round(scores, 3))
    print("Mean Balanced Accuracy:", np.round(np.mean(scores), 3))

    return scores


In [12]:
evaluate_with_smote_cv_rfoptimal(X,y)

Balanced Accuracy per fold: [0.919 0.927 0.934 0.885 0.862]
Mean Balanced Accuracy: 0.906


array([0.91929318, 0.92696924, 0.93395497, 0.88502762, 0.86230283])

In [13]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import make_scorer, balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from lightgbm import LGBMClassifier

def evaluate_with_smote_cv_lightgbm(X, y, test_size=0.2, n_splits=5, random_state=42):
    """
    Splits data, evaluates LightGBM with SMOTE + Stratified K-Fold CV,
    and trains a final model on the balanced training set.

    Args:
        X (pd.DataFrame): Feature matrix
        y (pd.Series): Target column
        test_size (float): Proportion for test split
        n_splits (int): Number of folds for cross-validation
        random_state (int): Random seed

    Returns:
        model (LGBMClassifier): Trained LightGBM model on SMOTE-balanced training data
        scores (list): Balanced accuracy scores across CV folds
        (X_train, X_test, y_train, y_test): Data splits
    """

    # --- 1. Train/test split with stratification ---
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_size,
        stratify=y,
        random_state=random_state
    )

    # --- 2. Define pipeline: SMOTE + LightGBM ---
    pipeline = Pipeline(steps=[
        ('smote', SMOTE(random_state=random_state)),
        ('lgbm', LGBMClassifier(
            n_estimators=500,
            learning_rate=0.05,
            max_depth=-1,
            class_weight='balanced',
            random_state=random_state,
            n_jobs=-1,
            verbose=-1
        ))
    ])

    # --- 3. Stratified CV evaluation ---
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    scorer = make_scorer(balanced_accuracy_score)
    scores = cross_val_score(pipeline, X_train, y_train, cv=skf, scoring=scorer, n_jobs=-1)

    print("Balanced Accuracy per fold:", np.round(scores, 3))
    print("Mean Balanced Accuracy:", np.round(np.mean(scores), 3))

    # --- 4. Train final model with SMOTE applied on training set ---
    smote = SMOTE(random_state=random_state)
    X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

    model = LGBMClassifier(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=-1,
        class_weight='balanced',
        random_state=random_state,
        n_jobs=-1,
        verbose=-1
    )

    model.fit(X_train_bal, y_train_bal)

    return model, scores, (X_train, X_test, y_train, y_test)


In [14]:
# # Run evaluation + final training
# model, scores, (X_train, X_test, y_train, y_test) = evaluate_with_smote_cv_lightgbm(X, y)

In [15]:
# import numpy as np
# import optuna
# from sklearn.model_selection import StratifiedKFold, cross_val_score
# from sklearn.metrics import make_scorer, balanced_accuracy_score
# from imblearn.pipeline import Pipeline
# from imblearn.over_sampling import SMOTE
# from lightgbm import LGBMClassifier

# def objective(trial, X, y, n_splits=5, random_state=42):
#     params = {
#         "objective": "multiclass",
#         "num_class": 3,
#         "n_estimators": 800,  # fixed during search
#         "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.15),
#         "num_leaves": trial.suggest_int("num_leaves", 16, 256, log=True),
#         "max_depth": trial.suggest_categorical("max_depth", [-1, 4, 6, 8, 10, 12, 16]),
#         "min_child_samples": trial.suggest_int("min_child_samples", 5, 60),
#         "min_split_gain": trial.suggest_float("min_split_gain", 0.0, 0.2),
#         "lambda_l1": trial.suggest_float("lambda_l1", 0.0, 1.0),
#         "lambda_l2": trial.suggest_float("lambda_l2", 0.0, 2.0),
#         "feature_fraction": trial.suggest_float("feature_fraction", 0.6, 1.0),
#         "bagging_fraction": trial.suggest_float("bagging_fraction", 0.6, 1.0),
#         "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
#         "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
#         "random_state": random_state,
#         "n_jobs": -1,
#         "verbose": -1,
#     }

#     pipe = Pipeline(steps=[
#         ("smote", SMOTE(random_state=random_state)),
#         ("lgbm", LGBMClassifier(**params))
#     ])

#     skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
#     scorer = make_scorer(balanced_accuracy_score)
#     scores = cross_val_score(pipe, X, y, cv=skf, scoring=scorer, n_jobs=-1)
#     return scores.mean()

# # Run study
# study = optuna.create_study(direction="maximize")
# study.optimize(lambda t: objective(t, X, y), n_trials=40, show_progress_bar=True)
# best_params = study.best_trial.params


In [16]:
# import numpy as np
# import pandas as pd
# from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
# from sklearn.metrics import make_scorer, balanced_accuracy_score
# from imblearn.over_sampling import SMOTE
# from imblearn.pipeline import Pipeline
# from lightgbm import LGBMClassifier

# def evaluate_with_smote_cv_lightgbm_optimal(X, y, test_size=0.2, n_splits=5, random_state=42):
#     """
#     Splits data, evaluates LightGBM with SMOTE + Stratified K-Fold CV,
#     and trains a final model on the balanced training set using the best tuned parameters.

#     Args:
#         X (pd.DataFrame): Feature matrix
#         y (pd.Series): Target column
#         test_size (float): Proportion for test split
#         n_splits (int): Number of folds for cross-validation
#         random_state (int): Random seed

#     Returns:
#         model (LGBMClassifier): Trained LightGBM model on SMOTE-balanced training data
#         scores (list): Balanced accuracy scores across CV folds
#         (X_train, X_test, y_train, y_test): Data splits
#     """

#     # --- 1. Train/test split with stratification ---
#     X_train, X_test, y_train, y_test = train_test_split(
#         X, y,
#         test_size=test_size,
#         stratify=y,
#         random_state=random_state
#     )

#     # --- 2. Best parameters from Optuna (Trial 27) ---
#     best_params = {
#         'learning_rate': 0.09518867841433679,
#         'num_leaves': 27,
#         'max_depth': 12,
#         'min_child_samples': 26,
#         'min_split_gain': 0.0637337805202833,
#         'lambda_l1': 0.6089094114371527,
#         'lambda_l2': 1.2021507701921763,
#         'feature_fraction': 0.8839452483534593,
#         'bagging_fraction': 0.8166691612094572,
#         'bagging_freq': 6,
#         'extra_trees': False,
#         'n_estimators': 800,           # still use tuned iterations
#         'objective': 'multiclass',
#         'num_class': 3,
#         'random_state': random_state,
#         'n_jobs': -1,
#         'verbose': -1
#     }

#     # --- 3. Stratified CV evaluation with SMOTE ---
#     pipeline = Pipeline(steps=[
#         ('smote', SMOTE(random_state=random_state)),
#         ('lgbm', LGBMClassifier(**best_params))
#     ])

#     skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
#     scorer = make_scorer(balanced_accuracy_score)
#     scores = cross_val_score(pipeline, X_train, y_train, cv=skf, scoring=scorer, n_jobs=-1)

#     print("Balanced Accuracy per fold:", np.round(scores, 3))
#     print("Mean Balanced Accuracy:", np.round(np.mean(scores), 3))

#     # --- 4. Train final model with SMOTE applied on training set ---
#     smote = SMOTE(random_state=random_state)
#     X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)

#     model = LGBMClassifier(**best_params)
#     model.fit(X_train_bal, y_train_bal)

#     return model, scores, (X_train, X_test, y_train, y_test)


In [17]:
# Run evaluation + final training
model, scores, (X_train, X_test, y_train, y_test) = evaluate_with_smote_cv_lightgbm_optimal(X, y)

NameError: name 'evaluate_with_smote_cv_lightgbm_optimal' is not defined

In [26]:
import pandas as pd
import matplotlib.pyplot as plt

def get_feature_importance(model, feature_names, top_n=20):
    """
    Extracts and plots feature importance from a trained LightGBM model.

    Args:
        model (LGBMClassifier): Trained LightGBM model
        feature_names (list): List of feature names
        top_n (int): Number of top features to display (default=20)

    Returns:
        pd.DataFrame: Feature importance sorted by importance
    """
    # Get feature importances (gain = how much each feature contributes to splits)
    importance = model.feature_importances_

    # Build DataFrame
    fi_df = pd.DataFrame({
        "Feature": feature_names,
        "Importance": importance
    }).sort_values(by="Importance", ascending=False)

    # Plot top features
    plt.figure(figsize=(8, 6))
    plt.barh(fi_df["Feature"].head(top_n)[::-1], fi_df["Importance"].head(top_n)[::-1])
    plt.xlabel("Importance (split count)")
    plt.ylabel("Feature")
    plt.title("LightGBM Feature Importance")
    plt.show()

    return fi_df


In [ ]:

# Get feature importance
fi = get_feature_importance(model, X.columns, top_n=20)

print(fi.head(20))


In [ ]:
# import shap
# import matplotlib.pyplot as plt
# import numpy as np

# def explain_with_shap_lightgbm(model, X, max_display=20, show_force_plot=False, sample_index=0):
#     """
#     Generates SHAP feature importance analysis for a trained LightGBM model.
    
#     Args:
#         model (LGBMClassifier): Trained LightGBM model
#         X (pd.DataFrame): Feature dataframe used during training
#         max_display (int): Max number of features to display in SHAP plots
#         show_force_plot (bool): Whether to show individual force plot (optional)
#         sample_index (int): Row index for local explanation (default: 0)
    
#     Returns:
#         shap_values (list[np.ndarray]): SHAP values for each class
#     """

#     # --- 1. Initialize SHAP Explainer ---
#     print("Initializing SHAP TreeExplainer...")
#     explainer = shap.TreeExplainer(model)

#     # --- 2. Compute SHAP Values ---
#     print("Computing SHAP values...")
#     shap_values = explainer.shap_values(X)

#     # --- 3. Global Interpretability ---
#     print("\n=== GLOBAL FEATURE IMPORTANCE ===")
#     plt.title("Mean |SHAP value| (Global Feature Importance)")
#     shap.summary_plot(shap_values, X, plot_type="bar", max_display=max_display)

#     print("\n=== DETAILED DISTRIBUTION (Beeswarm Plot) ===")
#     shap.summary_plot(shap_values, X, max_display=max_display)

#     # --- 4. Local Interpretability (Optional) ---
#     if show_force_plot:
#         print(f"\n=== LOCAL INTERPRETATION for sample index {sample_index} ===")
#         shap.initjs()
#         display(shap.force_plot(
#             explainer.expected_value[np.argmax(model.predict_proba(X.iloc[[sample_index]])[0])],
#             shap_values[np.argmax(model.predict_proba(X.iloc[[sample_index]])[0])][sample_index],
#             X.iloc[[sample_index]]
#         ))

#     # --- 5. Return SHAP values for further custom analysis ---
#     return shap_values


In [ ]:


# Generate SHAP explanations on the training data
shap_values = explain_with_shap_lightgbm(model, X_train)


In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.metrics import make_scorer, balanced_accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
import lightgbm as lgb


def evaluate_with_smote_cv_lightgbm_optimal(
    X, y, 
    test_size=0.2, 
    n_splits=5, 
    random_state=42,
    use_early_stopping=True,
    early_stopping_rounds=50,
    use_class_weights=False,
    smote_strategy='auto',
    verbose=True
):
    """
    Enhanced LightGBM training with SMOTE + Stratified K-Fold CV.
    
    Args:
        X (pd.DataFrame): Feature matrix
        y (pd.Series): Target column
        test_size (float): Proportion for test split
        n_splits (int): Number of folds for cross-validation
        random_state (int): Random seed
        use_early_stopping (bool): Whether to use early stopping for final model
        early_stopping_rounds (int): Number of rounds for early stopping
        use_class_weights (bool): Use class_weight='balanced' instead of/with SMOTE
        smote_strategy (str): SMOTE sampling strategy ('auto', 'not majority', 'all')
        verbose (bool): Print detailed output
    
    Returns:
        dict: Contains model, scores, splits, and diagnostics
    """
    # --- 1. Train/test split with stratification ---
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=test_size,
        stratify=y,
        random_state=random_state
    )
    
    if verbose:
        print("=" * 60)
        print("TRAINING CONFIGURATION")
        print("=" * 60)
        print(f"Train size: {len(X_train)}, Test size: {len(X_test)}")
        print(f"Class distribution (train): {dict(y_train.value_counts().sort_index())}")
        print(f"Class distribution (test): {dict(y_test.value_counts().sort_index())}")
        print()
    
    # --- 2. Best parameters from Optuna (Trial 27) ---
    best_params = {
        'learning_rate': 0.09518867841433679,
        'num_leaves': 27,
        'max_depth': 12,
        'min_child_samples': 26,
        'min_split_gain': 0.0637337805202833,
        'lambda_l1': 0.6089094114371527,
        'lambda_l2': 1.2021507701921763,
        'feature_fraction': 0.8839452483534593,
        'bagging_fraction': 0.8166691612094572,
        'bagging_freq': 6,
        'extra_trees': False,
        'n_estimators': 800,
        'objective': 'multiclass',
        'num_class': 3,
        'random_state': random_state,
        'n_jobs': -1,
        'verbose': -1
    }
    
    # Add class weights if requested
    if use_class_weights:
        best_params['class_weight'] = 'balanced'
        if verbose:
            print("Using class_weight='balanced'")
    
    # --- 3. Stratified CV evaluation with SMOTE ---
    if verbose:
        print("=" * 60)
        print("CROSS-VALIDATION EVALUATION")
        print("=" * 60)
    
    pipeline = Pipeline(steps=[
        ('smote', SMOTE(random_state=random_state, sampling_strategy=smote_strategy)),
        ('lgbm', LGBMClassifier(**best_params))
    ])
    
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    scorer = make_scorer(balanced_accuracy_score)
    scores = cross_val_score(pipeline, X_train, y_train, cv=skf, scoring=scorer, n_jobs=-1)
    
    if verbose:
        print(f"Balanced Accuracy per fold: {np.round(scores, 4)}")
        print(f"Mean Balanced Accuracy: {np.round(np.mean(scores), 4)} ± {np.round(np.std(scores), 4)}")
        print()
    
    # --- 4. Train final model with SMOTE applied on training set ---
    if verbose:
        print("=" * 60)
        print("FINAL MODEL TRAINING")
        print("=" * 60)
    
    smote = SMOTE(random_state=random_state, sampling_strategy=smote_strategy)
    X_train_bal, y_train_bal = smote.fit_resample(X_train, y_train)
    
    if verbose:
        print(f"After SMOTE - Train size: {len(X_train_bal)}")
        print(f"Class distribution (balanced): {dict(pd.Series(y_train_bal).value_counts().sort_index())}")
        print()
    
    model = LGBMClassifier(**best_params)
    
    # Fit with or without early stopping
    if use_early_stopping:
        callbacks = [lgb.early_stopping(early_stopping_rounds, verbose=False)]
        model.fit(
            X_train_bal, y_train_bal,
            eval_set=[(X_test, y_test)],
            callbacks=callbacks
        )
        if verbose and hasattr(model, 'best_iteration_'):
            print(f"Best iteration: {model.best_iteration_} (out of {best_params['n_estimators']})")
    else:
        model.fit(X_train_bal, y_train_bal)
    
    # --- 5. Feature importance ---
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    # --- 6. Test set evaluation ---
    y_pred = model.predict(X_test)
    test_balanced_acc = balanced_accuracy_score(y_test, y_pred)
    
    if verbose:
        print(f"\nTest Set Balanced Accuracy: {np.round(test_balanced_acc, 4)}")
        print("\nClassification Report (Test Set):")
        print(classification_report(y_test, y_pred))
        print("\nTop 10 Most Important Features:")
        print(feature_importance.head(10).to_string(index=False))
        print("=" * 60)
    
    # --- 7. Return comprehensive results ---
    return {
        'model': model,
        'cv_scores': scores,
        'cv_mean': np.mean(scores),
        'cv_std': np.std(scores),
        'test_balanced_accuracy': test_balanced_acc,
        'feature_importance': feature_importance,
        'splits': {
            'X_train': X_train,
            'X_test': X_test,
            'y_train': y_train,
            'y_test': y_test
        },
        'balanced_train_shape': X_train_bal.shape,
        'predictions': y_pred,
        'hyperparameters': best_params
    }


# Example usage:
"""
results = evaluate_with_smote_cv_lightgbm_optimal(
    X, y,
    test_size=0.2,
    n_splits=5,
    random_state=42,
    use_early_stopping=True,
    early_stopping_rounds=50,
    use_class_weights=False,
    verbose=True
)

# Access components
model = results['model']
cv_scores = results['cv_scores']
feature_importance = results['feature_importance']
X_train, X_test = results['splits']['X_train'], results['splits']['X_test']
y_train, y_test = results['splits']['y_train'], results['splits']['y_test']
"""

"\nresults = evaluate_with_smote_cv_lightgbm_optimal(\n    X, y,\n    test_size=0.2,\n    n_splits=5,\n    random_state=42,\n    use_early_stopping=True,\n    early_stopping_rounds=50,\n    use_class_weights=False,\n    verbose=True\n)\n\n# Access components\nmodel = results['model']\ncv_scores = results['cv_scores']\nfeature_importance = results['feature_importance']\nX_train, X_test = results['splits']['X_train'], results['splits']['X_test']\ny_train, y_test = results['splits']['y_train'], results['splits']['y_test']\n"

In [30]:
results = evaluate_with_smote_cv_lightgbm_optimal(
    X, y,
    test_size=0.2,
    n_splits=5,
    random_state=42,
    use_early_stopping=True,
    early_stopping_rounds=50,
    use_class_weights=False,
    verbose=True
)

TRAINING CONFIGURATION
Train size: 1692, Test size: 423
Class distribution (train): {1.0: 1318, 2.0: 234, 3.0: 140}
Class distribution (test): {1.0: 329, 2.0: 59, 3.0: 35}

CROSS-VALIDATION EVALUATION
Balanced Accuracy per fold: [0.8706 0.8914 0.9188 0.8916 0.9139]
Mean Balanced Accuracy: 0.8972 ± 0.0174

FINAL MODEL TRAINING
After SMOTE - Train size: 3954
Class distribution (balanced): {1.0: 1318, 2.0: 1318, 3.0: 1318}

Best iteration: 82 (out of 800)

Test Set Balanced Accuracy: 0.9261

Classification Report (Test Set):
              precision    recall  f1-score   support

         1.0       0.97      0.99      0.98       329
         2.0       0.91      0.85      0.88        59
         3.0       1.00      0.94      0.97        35

    accuracy                           0.96       423
   macro avg       0.96      0.93      0.94       423
weighted avg       0.96      0.96      0.96       423


Top 10 Most Important Features:
 feature  importance
    ASTV         602
    ALTV        

In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from typing import Optional, Tuple, Dict, List


def shap_feature_analysis(
    model,
    X_train: pd.DataFrame,
    X_test: pd.DataFrame,
    sample_size: int = 200,
    interaction_sample_size: int = 50,
    calculate_interactions: bool = True,
    low_importance_threshold: float = 0.01,
    low_interaction_threshold: float = 0.001,
    plot: bool = True,
    figsize: Tuple[int, int] = (15, 10)
) -> Dict:
    """
    Comprehensive SHAP analysis to identify features for removal based on 
    global importance and interaction effects.
    
    Args:
        model: Trained LightGBM/tree-based model
        X_train: Training feature matrix
        X_test: Test feature matrix
        sample_size: Number of samples for SHAP value calculation (reduced default)
        interaction_sample_size: Number of samples for interaction analysis (reduced default)
        calculate_interactions: Whether to calculate interactions (set False if memory issues)
        low_importance_threshold: Threshold for low importance (as fraction of max importance)
        low_interaction_threshold: Threshold for low interaction (as fraction of max interaction)
        plot: Whether to generate visualizations
        figsize: Figure size for plots
    
    Returns:
        dict: Comprehensive analysis results including features to drop
    """
    print("=" * 80)
    print("SHAP FEATURE IMPORTANCE ANALYSIS")
    print("=" * 80)
    
    # --- 1. Initialize SHAP explainer ---
    print("\n[1/5] Initializing SHAP explainer...")
    explainer = shap.TreeExplainer(model)
    
    # Sample data for efficiency
    X_train_sample = X_train.sample(min(sample_size, len(X_train)), random_state=42)
    X_test_sample = X_test.sample(min(sample_size, len(X_test)), random_state=42)
    
    # --- 2. Calculate SHAP values ---
    print(f"[2/5] Calculating SHAP values for {len(X_train_sample)} training samples...")
    shap_values_train = explainer(X_train_sample)
    
    print(f"      Calculating SHAP values for {len(X_test_sample)} test samples...")
    shap_values_test = explainer(X_test_sample)
    
    # For multiclass, take mean across classes
    if len(shap_values_train.shape) == 3:
        shap_values_train_mean = np.abs(shap_values_train.values).mean(axis=(0, 2))
        shap_values_test_mean = np.abs(shap_values_test.values).mean(axis=(0, 2))
    else:
        shap_values_train_mean = np.abs(shap_values_train.values).mean(axis=0)
        shap_values_test_mean = np.abs(shap_values_test.values).mean(axis=0)
    
    # --- 3. Global feature importance ---
    print("[3/5] Computing global feature importance...")
    
    feature_importance = pd.DataFrame({
        'feature': X_train.columns,
        'importance_train': shap_values_train_mean,
        'importance_test': shap_values_test_mean
    })
    feature_importance['importance_avg'] = (
        feature_importance['importance_train'] + feature_importance['importance_test']
    ) / 2
    feature_importance = feature_importance.sort_values('importance_avg', ascending=False)
    
    # Normalize to max importance
    max_importance = feature_importance['importance_avg'].max()
    feature_importance['importance_normalized'] = (
        feature_importance['importance_avg'] / max_importance
    )
    
    # Identify low importance features
    low_importance_features = feature_importance[
        feature_importance['importance_normalized'] < low_importance_threshold
    ]['feature'].tolist()
    
    print(f"\n   Found {len(low_importance_features)} low importance features")
    print(f"   (threshold: {low_importance_threshold * 100}% of max importance)")
    
    # --- 4. Feature interactions ---
    interaction_df = None
    interaction_strength = None
    low_interaction_features = []
    
    if calculate_interactions:
        print(f"\n[4/5] Calculating feature interactions for top features...")
        print(f"      (Using {interaction_sample_size} samples - this may take a moment)")
        
        try:
            # Sample for interaction analysis (computationally expensive)
            X_interaction_sample = X_train.sample(
                min(interaction_sample_size, len(X_train)), 
                random_state=42
            )
            
            # Calculate interactions for top 15 features only (reduced from 20)
            top_features = feature_importance.head(15)['feature'].tolist()
            X_interaction_top = X_interaction_sample[top_features]
            
            print(f"      Computing interactions for top {len(top_features)} features...")
            shap_interaction_values = explainer.shap_interaction_values(X_interaction_top)
            
            # For multiclass, average across classes
            if isinstance(shap_interaction_values, list):
                shap_interaction_values = np.mean(
                    [np.abs(arr) for arr in shap_interaction_values], 
                    axis=0
                )
            else:
                shap_interaction_values = np.abs(shap_interaction_values)
            
            # Average interaction matrix
            interaction_matrix = np.mean(shap_interaction_values, axis=0)
            
            # Create interaction dataframe
            interaction_df = pd.DataFrame(
                interaction_matrix,
                index=top_features,
                columns=top_features
            )
            
            # Get total interaction strength per feature
            # Exclude diagonal (self-interaction)
            np.fill_diagonal(interaction_matrix, 0)
            interaction_strength = pd.DataFrame({
                'feature': top_features,
                'total_interaction': interaction_matrix.sum(axis=1)
            }).sort_values('total_interaction', ascending=False)
            
            max_interaction = interaction_strength['total_interaction'].max()
            interaction_strength['interaction_normalized'] = (
                interaction_strength['total_interaction'] / max_interaction
            )
            
            # Identify features with low interaction
            low_interaction_features = interaction_strength[
                interaction_strength['interaction_normalized'] < low_interaction_threshold
            ]['feature'].tolist()
            
            print(f"\n   Found {len(low_interaction_features)} features with low interactions")
            print(f"   (threshold: {low_interaction_threshold * 100}% of max interaction)")
            
            # Clear memory
            del shap_interaction_values
            
        except Exception as e:
            print(f"\n   WARNING: Interaction calculation failed: {str(e)}")
            print("   Continuing with importance analysis only...")
            calculate_interactions = False
    else:
        print("\n[4/5] Skipping interaction calculation (calculate_interactions=False)")
    
    # --- 5. Recommendations for feature removal ---
    print("\n[5/5] Generating recommendations...")
    
    # Features that are BOTH low importance AND low interaction (if calculated)
    if calculate_interactions and low_interaction_features:
        features_to_drop_strict = list(
            set(low_importance_features) & set(low_interaction_features)
        )
    else:
        features_to_drop_strict = []
    
    # Features that are low importance (regardless of interaction)
    features_to_drop_moderate = low_importance_features
    
    # Create comprehensive report
    drop_recommendations = pd.DataFrame({
        'feature': X_train.columns,
        'importance_normalized': feature_importance.set_index('feature').loc[
            X_train.columns, 'importance_normalized'
        ].values,
        'is_low_importance': [f in low_importance_features for f in X_train.columns],
        'is_low_interaction': [
            f in low_interaction_features if (calculate_interactions and f in interaction_strength['feature'].values) else None 
            for f in X_train.columns
        ]
    }).sort_values('importance_normalized', ascending=True)
    
    # --- 6. Visualizations ---
    if plot:
        print("\nGenerating visualizations...")
        fig = plt.figure(figsize=figsize)
        
        # Plot 1: Feature Importance (Top 20)
        plt.subplot(2, 2, 1)
        top_20 = feature_importance.head(20)
        plt.barh(range(len(top_20)), top_20['importance_avg'])
        plt.yticks(range(len(top_20)), top_20['feature'])
        plt.xlabel('Mean |SHAP value|')
        plt.title('Top 20 Features by SHAP Importance')
        plt.gca().invert_yaxis()
        
        # Plot 2: Feature Importance (Bottom 20)
        plt.subplot(2, 2, 2)
        bottom_20 = feature_importance.tail(20)
        colors = ['red' if f in features_to_drop_strict else 'orange' 
                  if f in features_to_drop_moderate else 'gray' 
                  for f in bottom_20['feature']]
        plt.barh(range(len(bottom_20)), bottom_20['importance_avg'], color=colors)
        plt.yticks(range(len(bottom_20)), bottom_20['feature'])
        plt.xlabel('Mean |SHAP value|')
        plt.title('Bottom 20 Features (Red=Drop Strict, Orange=Drop Moderate)')
        plt.axvline(x=max_importance * low_importance_threshold, 
                   color='red', linestyle='--', alpha=0.5, label='Threshold')
        plt.legend()
        plt.gca().invert_yaxis()
        
        # Plot 3: Interaction Heatmap
        if calculate_interactions and interaction_df is not None:
            plt.subplot(2, 2, 3)
            sns.heatmap(
                interaction_df.head(10).iloc[:, :10], 
                cmap='YlOrRd', 
                cbar_kws={'label': 'Interaction Strength'}
            )
            plt.title('Feature Interaction Heatmap (Top 10 Features)')
            plt.xticks(rotation=45, ha='right')
            plt.yticks(rotation=0)
        else:
            plt.subplot(2, 2, 3)
            plt.text(0.5, 0.5, 'Interaction analysis\nnot calculated', 
                    ha='center', va='center', fontsize=12)
            plt.axis('off')
        
        # Plot 4: SHAP Summary Plot
        plt.subplot(2, 2, 4)
        shap.summary_plot(
            shap_values_test, 
            X_test_sample, 
            max_display=20,
            show=False
        )
        plt.title('SHAP Summary Plot (Test Set)')
        
        plt.tight_layout()
        plt.show()
    
    # --- 7. Print summary ---
    print("\n" + "=" * 80)
    print("SUMMARY & RECOMMENDATIONS")
    print("=" * 80)
    print(f"\nTotal features: {len(X_train.columns)}")
    print(f"\nFeatures to DROP (strict - low importance AND low interaction):")
    print(f"  Count: {len(features_to_drop_strict)}")
    if features_to_drop_strict:
        print(f"  Features: {features_to_drop_strict[:10]}")
        if len(features_to_drop_strict) > 10:
            print(f"  ... and {len(features_to_drop_strict) - 10} more")
    
    print(f"\nFeatures to CONSIDER dropping (moderate - low importance only):")
    print(f"  Count: {len(features_to_drop_moderate)}")
    if features_to_drop_moderate:
        print(f"  Features: {features_to_drop_moderate[:10]}")
        if len(features_to_drop_moderate) > 10:
            print(f"  ... and {len(features_to_drop_moderate) - 10} more")
    
    print(f"\nTop 5 most important features:")
    for idx, row in feature_importance.head(5).iterrows():
        print(f"  {row['feature']}: {row['importance_normalized']:.4f}")
    
    if calculate_interactions and interaction_strength is not None:
        print(f"\nTop 5 features by interaction strength:")
        for idx, row in interaction_strength.head(5).iterrows():
            print(f"  {row['feature']}: {row['interaction_normalized']:.4f}")
    
    print("\n" + "=" * 80)
    
    # --- 8. Return comprehensive results ---
    return {
        'feature_importance': feature_importance,
        'interaction_strength': interaction_strength,
        'interaction_matrix': interaction_df,
        'drop_recommendations': drop_recommendations,
        'features_to_drop_strict': features_to_drop_strict,
        'features_to_drop_moderate': features_to_drop_moderate,
        'low_importance_features': low_importance_features,
        'low_interaction_features': low_interaction_features,
        'shap_values_train': shap_values_train,
        'shap_values_test': shap_values_test,
        'thresholds': {
            'importance': low_importance_threshold,
            'interaction': low_interaction_threshold
        }
    }


# Example usage:
"""
# After training your model with the enhanced function:
results = evaluate_with_smote_cv_lightgbm_optimal(X, y)
model = results['model']
X_train = results['splits']['X_train']
X_test = results['splits']['X_test']

# Option 1: Quick analysis without interactions (if memory issues)
shap_results = shap_feature_analysis(
    model=model,
    X_train=X_train,
    X_test=X_test,
    sample_size=200,
    calculate_interactions=False,  # Skip interactions to save memory
    low_importance_threshold=0.01,
    plot=True
)

# Option 2: Full analysis with interactions (smaller samples)
shap_results = shap_feature_analysis(
    model=model,
    X_train=X_train,
    X_test=X_test,
    sample_size=200,
    interaction_sample_size=50,  # Very small sample for interactions
    calculate_interactions=True,
    low_importance_threshold=0.01,
    plot=True
)

# Get features to drop
features_to_drop = shap_results['features_to_drop_moderate']  # Based on importance only

# Retrain model without these features
X_reduced = X.drop(columns=features_to_drop)
results_reduced = evaluate_with_smote_cv_lightgbm_optimal(X_reduced, y)
"""

"\n# After training your model with the enhanced function:\nresults = evaluate_with_smote_cv_lightgbm_optimal(X, y)\nmodel = results['model']\nX_train = results['splits']['X_train']\nX_test = results['splits']['X_test']\n\n# Option 1: Quick analysis without interactions (if memory issues)\nshap_results = shap_feature_analysis(\n    model=model,\n    X_train=X_train,\n    X_test=X_test,\n    sample_size=200,\n    calculate_interactions=False,  # Skip interactions to save memory\n    low_importance_threshold=0.01,\n    plot=True\n)\n\n# Option 2: Full analysis with interactions (smaller samples)\nshap_results = shap_feature_analysis(\n    model=model,\n    X_train=X_train,\n    X_test=X_test,\n    sample_size=200,\n    interaction_sample_size=50,  # Very small sample for interactions\n    calculate_interactions=True,\n    low_importance_threshold=0.01,\n    plot=True\n)\n\n# Get features to drop\nfeatures_to_drop = shap_results['features_to_drop_moderate']  # Based on importance only\n\n

In [1]:
# After training your model
# results = evaluate_with_smote_cv_lightgbm_optimal(X, y)
model = results['model']
X_train = results['splits']['X_train']
X_test = results['splits']['X_test']

# Run SHAP analysis
shap_results = shap_feature_analysis(
    model=model,
    X_train=X_train,
    X_test=X_test,
    low_importance_threshold=0.01,  # Features with <1% of max importance
    plot=True
)

# Get recommended features to drop
features_to_drop = shap_results['features_to_drop_strict']

# Retrain without these features
X_reduced = X.drop(columns=features_to_drop)
results_new = evaluate_with_smote_cv_lightgbm_optimal(X_reduced, y)

NameError: name 'results' is not defined

lol
